# <u>This is our Mini project for the course "Fundamental Data Concepts" (4DACF) at SUPINFO Lyon :</u>

## <u>Evaluation Project - Data Processing and Visualization :</u>

## <u>Project Objective :</u>
You will design a complete data processing pipeline that includes several key steps: anonymization, transformation, cleaning, and data visualization.


The goal is to leverage multiple technologies to produce a high-quality pipeline that adheres to best practices.

This project must be carried out in groups of up to three students.

## <u>Contexte :</u>
A fictional e-commerce company aims to leverage its customer and transaction data while complying with GDPR regulations.

The company has a dataset containing sensitive information and seeks to obtain:

- [ ] An automated pipeline for anonymizing, transforming, and cleaning the data in python.
- [ ] A final output optimized for direct use in Power BI.

## <u>BONUS :</u>

- [ ] A set of visualizations in Python that provide insights into the data.

# <u>Step 1:</u> Pipeline Preparation: Python code for anonymization, cleaning, and transformation

In [ ]:
from calendar import month

# Importing the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import hashlib

## 1. Exploratory Data Analysis (EDA)

The first step in any data processing pipeline is to understand the data. This involves exploring the data to identify patterns, trends, and potential issues. EDA is a critical step that helps data engineer understand the data and make informed decisions about how to process it.

In [ ]:
# Load the dataset : Mini_Projet_Evaluation.csv

dataset_file_path: str = 'data/raw/Mini_Projet_Evaluation.csv'
dataset = pd.read_csv(dataset_file_path)

In [ ]:
# Explore the data : Structure of the data, data types, etc.

# Display the first few rows of the dataset

print(f"The dataset head is : \n {dataset.head()}") #to see a quick view of the dataset

In [ ]:
# Display the infos of the dataset

print(f"The dataset info is : \n {dataset.info()}") #to see the structure of the dataset

In [ ]:
# Display the shape of the dataset

print(f"This dataset have {dataset.shape[0]} entries and {dataset.shape[1]} columns.") #to see the number of rows and columns in resume, and to see the eventual problems on the dataset

In [ ]:
# Display the descriptive statistics of the dataset

print(f"The dataset describe is : \n {dataset.describe()}") #to see the statistical summary of the dataset, to see the eventual problems on the dataset (outliers, etc.)

In [ ]:
# Display the data types in the dataset

print(f"The dataset data types are : \n {dataset.dtypes}") #to see the data types of the dataset

## 2. Data Cleaning

Data cleaning is the process of identifying and correcting errors in the data. This step is essential for ensuring the quality of the data and the accuracy of the analysis. Data cleaning involves several key tasks, including:
- [X] Handling missing values
- [X] Removing duplicates
- [X] Correcting errors
- [X] Correcting incoherent values
- [X] Standardizing data
- [X] Handling outliers
- [X] Standardizing data types


In [ ]:
# Data Cleaning : Cleaning the dataset

# Handling missing values

missing_values_count = dataset.isnull().sum()
print(f"The missing values count is : \n {missing_values_count}") #to see the number of missing values in each column of the dataset

In [ ]:
# Removing duplicates

duplicates_values_count = dataset.duplicated().sum()
print(f"The number of duplicates in the dataset is : {duplicates_values_count}") #to see the number of duplicates in the dataset

In [ ]:
# Correcting errors

columns_to_verify = ['Sexe', 'CatégorieProduitPréféré', 'AvisClient', 'AbonnementNewsletter', 'TypePaiementFavori', 'StatutCompte'] #to see the columns that we want to verify

for col in columns_to_verify:
    print(f"\n🔹 {col} : {dataset[col].nunique()} unique values") #to see the number of unique values in each column of the dataset
    print(dataset[col].unique())  # to see the unique values in each column of the dataset

In [ ]:
# Correcting Incoherent values

# Incoherence detection
incoherence_achats = dataset[(dataset['NombreAchats'] == 0) & (dataset['MontantTotalAchats'] > 0)]
incoherence_panier = dataset[(dataset['MontantTotalAchats'] == 0) & (dataset['PanierMoyen'] > 0)]
remboursements_invalides = dataset[dataset['MontantTotalRemboursé'] > dataset['MontantTotalAchats']]

print(f"📌 Nombre de pages incohérentes (NombreAchats = 0 & MontantTotalAchats > 0) : {len(incoherence_achats)}")
print(f"📌 Nombre de pages incohérentes (MontantTotalAchats = 0 & PanierMoyen > 0) : {len(incoherence_panier)}")
print(f"📌 Nombre d'enregistrements où le MontantTotalRemboursé > MontantTotalAchats :, {len(remboursements_invalides)}")

# Correcting with mean function
dataset.loc[(dataset['NombreAchats'] == 0) & (dataset['MontantTotalAchats'] > 0), 'NombreAchats'] = dataset['NombreAchats'].median()
dataset.loc[(dataset['FréquenceAchatMensuel'] == 0) & (dataset['PanierMoyen'] > 0), 'FréquenceAchatMensuel'] = dataset['FréquenceAchatMensuel'].median()
dataset.loc[(dataset['MontantTotalAchats'] == 0) & (dataset['PanierMoyen'] > 0), 'PanierMoyen'] = 0  # Logic : if the total amount of purchases is 0, the average basket is 0

# deletion of the incoherent records
df = dataset[dataset['MontantTotalRemboursé'] <= dataset['MontantTotalAchats']]

In [ ]:
# Standardizing data

# Standardizing the float columns
columns_to_round = ['MontantTotalAchats', 'SoldeCompte', 'PanierMoyen', 'MontantTotalRemboursé'] #to see the columns that we want to round
for col in columns_to_round:
    dataset[col] = dataset[col].apply(lambda x: round(x, 2)) #to round the values in each column of the dataset

# Standardizing the countries and cities columns
standardize_name_columns = ['Pays', 'Ville'] #to see the columns that we want to standardize
for col in standardize_name_columns:
    dataset[col] = dataset[col].str.title() #to standardize the values in each column of the dataset

# Standardizing the date columns
standardize_date_columns = ['DateNaissance', 'DernierAchat', 'DateExpirationCarte'] #to see the columns that we want to standardize
def standardise_date(date):
    date = str(date).strip()
    if re.match(r"^\d{4}-\d{2}-\d{2}$", date):
        return date  # yyyy-mm-dd -> yyyy-mm-dd
    elif re.match(r"^\d{2}/\d{2}/\d{4}$", date):
        match = re.match(r"^(\d{2})/(\d{2})/(\d{4})$", date)
        return f"{match.group(3)}-{match.group(1)}-{match.group(2)}"  # mm/dd/yyyy -> yyyy-mm-dd
    elif re.match(r"^(\d{2})/(\d{2})$", date):
        match = re.match(r"^(\d{2})/(\d{2})$", date)
        return f"20{match.group(2)}-{match.group(1)}-01"  # mm/dd -> yyyy-mm-dd
    elif re.match(r"^\d{2}-\d{2}-\d{4}$", date):
        match = re.match(r"^(\d{2})-(\d{2})-(\d{4})$", date)
        return f"{match.group(3)}-{match.group(2)}-{match.group(1)}"  # dd-mm-yyyy -> yyyy-mm-dd
    elif re.match(r"^\d{2}-\d{2}-\d{2}$", date):
        match = re.match(r"^(\d{2})-(\d{2})-(\d{2})$", date)
        return f"20{match.group(3)}-{match.group(2)}-{match.group(1)}"  # dd-mm-yy -> yyyy-mm-dd
    return date

for col in standardize_date_columns:
    dataset[col] = dataset[col].apply(standardise_date) #to standardize the values in each column of the dataset

In [ ]:
# Handling outliers (Outliers detection)

# Quartiles and IQR for outliers detection
Q1 = dataset['PanierMoyen'].quantile(0.25)
Q3 = dataset['PanierMoyen'].quantile(0.75)
IQR = Q3 - Q1

# bounds definition for outliers detection
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# outliers identification
outliers = dataset[(dataset['PanierMoyen'] < lower_bound) | (dataset['PanierMoyen'] > upper_bound)]
print(f"Outliers numbers find : {len(outliers)}")
display(outliers)

# PanierMoyen graphic representation

# Boxplot of PanierMoyen
plt.figure(figsize=(10,5))
sns.boxplot(x=dataset['PanierMoyen'])
plt.title("Boxplot of PanierMoyen")
plt.show()

# Scatterplot of NombreAchats and PanierMoyen
plt.figure(figsize=(10,5))
sns.scatterplot(x=dataset['NombreAchats'], y=dataset['PanierMoyen'])
plt.title("Scatterplot of NombreAchats and PanierMoyen")
plt.xlabel("Nombre d'Achats")
plt.ylabel("Panier Moyen")
plt.show()

In [ ]:
# Standardizing data types

# data types detection
print(f"The dataset data types before cleaning are : \n {dataset.dtypes}") #to see the data types of the dataset

# Standardizing the data types of the columns
dataset['DateNaissance'] = pd.to_datetime(dataset['DateNaissance']) #to convert the data type of the column to datetime
dataset['DernierAchat'] = pd.to_datetime(dataset['DernierAchat']) #to convert the data type of the column to datetime

print(f"The dataset data types after cleaning are : \n {dataset.dtypes}") #to see the data types of the dataset

## 3. Data Transformation (Anonymization, pseudonomization, columns selection etc...)
Data transformation is the process of converting raw data into a format that is suitable for analysis. This step involves several key tasks, including:
- [X] Anonymization
- [X] Pseudonymization
- [X] Aggregation
- [X] Data reduction
- [X] Data Addition

In [ ]:
# Anonymization

# Nom et Prénom : Anonymisation
def anonymize_name(name):
    # return hashlib.sha256(name.encode()).hexdigest()[:8]
    return hashlib.md5(name.encode()).hexdigest()

dataset['Nom'] = dataset['Nom'].astype(str).apply(anonymize_name)
dataset['Prénom'] = dataset['Prénom'].astype(str).apply(anonymize_name)

# Adresse email : Anonymisation
def anonymize_email(email):
    username = email.split('@')[0]
    hashed_username = hashlib.md5(username.encode()).hexdigest()
    return f"{hashed_username}@masked.com"

dataset['Email'] = dataset['Email'].astype(str).apply(anonymize_email)

# Adresse : Anonymisation
dataset['Adresse'] = "masked"

# Téléphone : Anonymisation
dataset['Téléphone'] = "masked"

In [ ]:
# Pseudonymization

# Pseudonymization of the NumeroCarteCredit column
def pseudonymize_credit_card(card_number):
    return f"**** **** **** {card_number[-4:]}"  # to keep only the last 4 digits of the credit card number

dataset['NuméroCarteCrédit'] = dataset['NuméroCarteCrédit'].astype(str).apply(pseudonymize_credit_card)

# Pseudonymization of the DateExpirationCarte column
def pseudonymize_credit_card_expiration(expiration_date):
    return expiration_date[-2:]  # to keep only the last 2 digits of the expiration date

dataset['DateExpirationCarte'] = dataset['DateExpirationCarte'].astype(str).apply(pseudonymize_credit_card_expiration)

# Pseudonymization of the code postal column
def pseudonymize_postal_code(postal_code):
    return postal_code[:2] + "***"  # to keep only the first 2 digits of the postal code

In [ ]:
# Aggregation

# Aggregation of the age column
bins = [0, 18, 25, 35, 45, 55, 65, 100]  # age gap definition
labels = ["0-18", "19-25", "26-35", "36-45", "46-55", "56-65", "65+"]

dataset['Âge'] = pd.cut(dataset['Âge'], bins=bins, labels=labels)

In [ ]:
# Data Reduction

# Columns_to_delete = ['CodePostal', 'Nom', 'Prénom', 'Email', 'Adresse', 'Téléphone', 'DateNaissance', 'NuméroCarteCrédit', 'DateExpirationCarte', 'SoldeCompte']
# dataset.drop(columns=Columns_to_delete, inplace=True) #to delete the columns that we don't need anymore

In [ ]:
# Data Addition
# Addition of columns



## 4. Validation
Data validation is the process of ensuring that the data is accurate, complete, and consistent. This step involves several key tasks, including:
- [X] Data profiling

In [ ]:
# Data profiling after cleaning and transformation

# Display the first few rows of the dataset after cleaning and transformation

print(f"The clean dataset head is : \n {dataset.head()}") #to see a quick view of the dataset

In [ ]:
# Display the infos of the dataset

print(f"The dataset info is : \n {dataset.info()}") #to see the structure of the dataset

In [ ]:
# Display the shape of the dataset

print(f"This dataset have {dataset.shape[0]} entries and {dataset.shape[1]} columns after the cleaning and the transformation.")

In [ ]:
# Description of the dataset after cleaning and transformation
print(f"The dataset describe is : \n {dataset.describe()}") #to see the statistical summary of the dataset

In [ ]:
# Data types verification
print(f"The dataset data types are : \n {dataset.dtypes}") #to see the data types of the dataset

In [ ]:
# missing values verification
print(dataset.isnull().sum()) #to see the number of missing values in each column of the dataset

## 5. Data Export (To CSV)
The final step in the data processing pipeline is to export the cleaned and transformed data to a file format that can be used for analysis. This step involves exporting the data to a CSV or Excel file, which can then be imported into a data visualization tool for further analysis.

In [ ]:
# Exporting the cleaned and transformed data to a CSV file
export_path = "data/processed/Mini_Projet_Evaluation_Cleaned_transform.csv"

dataset.to_csv(export_path, index=False, encoding='utf-8')
print(f"The file was exported successfully : {export_path}")

## 6. Data Visualization (BONUS)

Data visualization is the process of representing data graphically to help data engineers and analysts understand the data and identify patterns and trends. Data visualization is a critical step in the data processing pipeline, as it helps to communicate the results of the analysis to stakeholders and decision-makers. Data visualization involves several key tasks, including: